In [ ]:
import json

file = []
with open("newdataset2.json", "r") as f:
    for line in f:
        file.append(json.loads(line))
print(file[1])

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

In [ ]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

In [ ]:
# Train the model
trainer_stats = trainer.train()

In [ ]:
import gc

# Eliminar el trainer y las estadísticas para liberar RAM
if "trainer" in locals():
    del trainer
if "trainer_stats" in locals():
    del trainer_stats

# Forzar al recolector de basura de Python
gc.collect()

# Limpiar la caché de la GPU
torch.cuda.empty_cache()

# Esto casi no consume RAM y asegura tu trabajo
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {
        "role": "user",
        "content": "Tutor, ¿cuál es la mejor forma de abordar Two Sum?"
    },
    {
        "role": "assistant",
        "content": "¡Claro que sí! Te explico Two Sum como si fuera una receta de cocina. Imagina que estás en el supermercado con una lista de precios y necesitas encontrar dos productos que sumen exactamente el presupuesto que llevas.\n\n**Paso a paso:**\n1. **Preparar tus ingredientes:** Creamos un \"diccionario de memoria\" (un `map`) donde iremos anotando cada número que veamos y en qué posición está.\n2. **Revisar la lista:** Vamos recorriendo tu lista de números uno por uno, fijándonos tanto en el valor como en su posición.\n3. **Calcular lo que falta:** Para cada número, calculamos cuánto dinero nos faltaría para llegar al objetivo (esto es `objetivo - número_actual`).\n4. **Buscar en la memoria:** Revisamos si ese \"dinero que falta\" ya lo vimos antes en nuestro diccionario. ¡Si está ahí, genial! Encontramos la pareja perfecta.\n5. **Tomar nota:** Si no lo encontramos, simplemente anotamos este número en nuestro diccionario para recordarlo más tarde.\n\n**La receta escrita (código):**\n```python\ndef encontrar_pareja(nums, objetivo):\n    memoria = {}  # Nuestro diccionario para recordar\n    \n    for posicion, numero in enumerate(nums):\n        complemento = objetivo - numero\n        \n        if complemento in memoria:\n            return [memoria[complemento], posicion]\n        \n        memoria[numero] = posicion\n    \n    return []  # Si no encontramos nada\n```\n\n**Tiempo de preparación:** Solo necesitamos revisar la lista una vez (O(n)), y usamos un poco de espacio para nuestro diccionario (también O(n)).\n\n¡Y listo! Así de sencillo es encontrar esa pareja perfecta de números que sumen exactamente lo que necesitas."
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)